# 3.1.1 Mighty MLP, the most basic form of neural network
### On tabular data

### requirements
```
python 3.6

numpy == '1.14.3'
pandas == '0.23.0'
tensorflow == '1.8.0'
keras == '2.2.0'
```
Other versions of above library will probably work

We are going to start with this dataset:[Simplified Human Activity Recognition w/Smartphone](https://www.kaggle.com/mboaglio/simplifiedhuarus)

```
Content
Abstract: Human Activity Recognition database built from the recordings of 30 subjects performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors.

Data Set Characteristics: Multivariate, Time-Series

Data Set Information:

The experiments have been carried out with a group of 30 volunteers within an age bracket of 19-48 years. Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (Samsung Galaxy S II) on the waist. Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity at a constant rate of 50Hz. The experiments have been video-recorded to label the data manually. The obtained dataset has been randomly partitioned into two sets, where 70% of the volunteers was selected for generating the training data and 30% the test data.

The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used. From each window, a vector of features was obtained by calculating variables from the time and frequency domain.
```

Actually I didn't finish reading the part I have posted above...

In [1]:
import pandas as pd
import numpy as np

You can download the train and test datasets also from here, and place it in /data

http://45.76.223.58/human_phone_train.csv

http://45.76.223.58/human_phone_test.csv

In [2]:
train = pd.read_csv("/data/human_phone_train.csv")
test = pd.read_csv("/data/human_phone_test.csv")
# or run this:
# train = pd.read_csv("http://45.76.223.58/human_phone_train.csv")
# test = pd.read_csv("http://45.76.223.58/human_phone_test.csv")

In [3]:
train.values.shape

(3609, 563)

In [4]:
train_y = train.values[...,1]

In [5]:
train_y

array(['STANDING', 'STANDING', 'STANDING', ..., 'WALKING_UPSTAIRS',
       'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'], dtype=object)

Use set to remove the duplicates and see the categories

In [6]:
cates = list(set(train_y.tolist()))

In [7]:
cates

['LAYING',
 'SITTING',
 'WALKING_UPSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING',
 'STANDING']

In [8]:
cates2idx = dict((v,k) for k,v in enumerate(cates))

A mapping dictionary

In [9]:
cates2idx

{'LAYING': 0,
 'SITTING': 1,
 'WALKING_UPSTAIRS': 2,
 'WALKING_DOWNSTAIRS': 3,
 'WALKING': 4,
 'STANDING': 5}

In [10]:
train_y_arr = np.vectorize(lambda x:cates2idx[x])(train_y)
train_y_arr

array([5, 5, 5, ..., 2, 2, 3])

### Transform to onehot encoding

In [11]:
np.eye(len(cates))

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [12]:
train_y_oh = np.eye(len(cates))[train_y_arr]

In [13]:
train_y_oh

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [14]:
train_x_arr  = train.values[...,2:]
train_x_arr

array([[0.27899999999999997, -0.0196, -0.11, ..., -0.852, 0.182, -0.043],
       [0.27699999999999997, -0.0127, -0.10300000000000001, ..., -0.852,
        0.188, -0.0347],
       [0.27699999999999997, -0.0147, -0.107, ..., -0.85,
        0.18899999999999997, -0.0351],
       ...,
       [0.284, -0.00796, -0.11900000000000001, ..., -0.657, 0.272, 0.183],
       [0.207, 0.0246, -0.10400000000000001, ..., -0.657, 0.267, 0.188],
       [0.39299999999999996, -0.0178, -0.0902, ..., -0.807, 0.19,
        0.11800000000000001]], dtype=object)

In [15]:
train_x_arr.shape, train_y_oh.shape

((3609, 561), (3609, 6))

## Building Model

In [16]:
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam

/Users/salvor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Pure Linear

In [17]:
ipt = Input((561,))
x = Dense(len(cates),activation="softmax")(ipt)
model = Model(ipt,x)

In [18]:
model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

In [19]:
model.fit(train_x_arr,train_y_oh,batch_size=256,epochs=15,validation_split=.3,shuffle=True)

Train on 2526 samples, validate on 1083 samples
Epoch 1/15
2526/2526 [==============================] - 1s 340us/step - loss: 1.6585 - acc: 0.2629 - val_loss: 1.4660 - val_acc: 0.3232
Epoch 2/15
2526/2526 [==============================] - 1s 220us/step - loss: 1.2705 - acc: 0.4640 - val_loss: 1.1537 - val_acc: 0.5549
Epoch 3/15
2526/2526 [==============================] - 1s 214us/step - loss: 1.0586 - acc: 0.6049 - val_loss: 1.0198 - val_acc: 0.6602
Epoch 4/15
2526/2526 [==============================] - 1s 209us/step - loss: 0.9361 - acc: 0.7110 - val_loss: 0.9082 - val_acc: 0.7424
Epoch 5/15
2526/2526 [==============================] - 1s 218us/step - loss: 0.8397 - acc: 0.7724 - val_loss: 0.8339 - val_acc: 0.7747
Epoch 6/15
2526/2526 [==============================] - 1s 227us/step - loss: 0.7677 - acc: 0.8076 - val_loss: 0.7705 - val_acc: 0.8172
Epoch 7/15
2526/2526 [==============================] - 1s 242us/step - loss: 0.7065 - acc: 0.8373 - val_loss: 0.7162 - val_acc: 0.8403


### Neural Network

In [20]:
ipt = Input((561,))
x = Dense(1000,activation="relu")(ipt)
x = Dense(len(cates),activation="softmax")(x)
model = Model(ipt,x)

#### About "relu"

"relu" is a frequently used activation layer for neural network

relu helps the model to [learn the xor gate](https://www.beyondthelines.net/machine-learning/neural-network/).  With this neural network can figure out automatically, by learning, a full spectrum of [logic gates](https://en.wikipedia.org/wiki/Logic_gate).

And mlp is like hundreds/thousands of logic gates assembled together. It's some sort of powerful organ to learn,summarize and figure out reasons from the historical pattern between input feature and output results. 

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 561)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              562000    
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 6006      
Total params: 568,006
Trainable params: 568,006
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

In [23]:
model.fit(train_x_arr,train_y_oh,batch_size=256,epochs=15,validation_split=.3,shuffle=True)

Train on 2526 samples, validate on 1083 samples
Epoch 1/15
2526/2526 [==============================] - 1s 419us/step - loss: 1.2296 - acc: 0.5344 - val_loss: 0.6912 - val_acc: 0.7747
Epoch 2/15
2526/2526 [==============================] - 1s 293us/step - loss: 0.4883 - acc: 0.8298 - val_loss: 0.4162 - val_acc: 0.8550
Epoch 3/15
2526/2526 [==============================] - 1s 305us/step - loss: 0.3147 - acc: 0.8975 - val_loss: 0.3628 - val_acc: 0.8569
Epoch 4/15
2526/2526 [==============================] - 1s 298us/step - loss: 0.2424 - acc: 0.9192 - val_loss: 0.2658 - val_acc: 0.9141
Epoch 5/15
2526/2526 [==============================] - 1s 297us/step - loss: 0.2034 - acc: 0.9303 - val_loss: 0.2356 - val_acc: 0.9298
Epoch 6/15
2526/2526 [==============================] - 1s 288us/step - loss: 0.1720 - acc: 0.9466 - val_loss: 0.2352 - val_acc: 0.9141
Epoch 7/15
2526/2526 [==============================] - 1s 301us/step - loss: 0.1536 - acc: 0.9525 - val_loss: 0.2136 - val_acc: 0.9271


### Deeper Neural Network

In [24]:
ipt = Input((561,))
x = Dense(1000,activation="relu")(ipt)
x = Dense(1000,activation="relu")(x)
x = Dense(1000,activation="relu")(x)
x = Dense(len(cates),activation="softmax")(x)
model = Model(ipt,x)

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 561)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              562000    
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_6 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 6006      
Total params: 2,570,006
Trainable params: 2,570,006
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_x_arr,train_y_oh,batch_size=256,epochs=15,validation_split=.3,shuffle=True)

Overfiting is happending, we can try a mechanism to handle over fitting

### Add dropouts

Dropouts are not very complex mechanism but sure is very useful, to help understand dropout, I found google's deepmind's explain on [nueron deletion](https://deepmind.com/blog/understanding-deep-learning-through-neuron-deletion/) very helpful.

Dropout is just a mechanism(an nn layer actually) to do neuron deletion ramdomly during training

In [ ]:
do_ratio = .3 # dropout ratio
ipt = Input((561,))
x = Dense(1000,activation="relu")(ipt)
x = Dropout(do_ratio)(x)
x = Dense(1000,activation="relu")(x)
x = Dropout(do_ratio)(x)
x = Dense(1000,activation="relu")(x)
x = Dropout(do_ratio)(x)
x = Dense(len(cates),activation="softmax")(x)
model = Model(ipt,x)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_x_arr,train_y_oh,batch_size=256,epochs=15,validation_split=.3,shuffle=True)

## Sequetial Format 

Aside from the "functional API" of keras you've seen above, you can also do "Sequential" model if your model is a long one way pipeline

In [ ]:
do_ratio = .3
model = Sequential([
    Dense(1000, input_shape=(561,)),
    Activation('relu'),
    Dropout(do_ratio),
    Dense(1000),
    Activation('relu'),
    Dropout(do_ratio),
    Dense(1000),
    Activation('relu'),
    Dropout(do_ratio),
    Dense(len(cates)),
    Activation('softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(train_x_arr,train_y_oh,batch_size=256,epochs=15,validation_split=.3,shuffle=True)

### Saving and loading keras model

We can save the model to a h5 file

In [ ]:
model.save_weights("/data/mlp.0.0.1.h5")

We can load the model from a h5 file

In [ ]:
do_ratio = .3
model_2 = Sequential([
    Dense(1000, input_shape=(561,)),
    Activation('relu'),
    Dropout(do_ratio),
    Dense(1000),
    Activation('relu'),
    Dropout(do_ratio),
    Dense(1000),
    Activation('relu'),
    Dropout(do_ratio),
    Dense(len(cates)),
    Activation('softmax'),
])

In [ ]:
model_2.load_weights("/data/mlp.0.0.1.h5")

### Prediction with keras model

In [ ]:
test_array = test.values[...,1:]

In [ ]:
result_array = model_2.predict(test_array)

In [ ]:
result_array, result_array.shape

In [ ]:
result_idx = result_array.argmax(axis=-1)

In [ ]:
result_idx[:20]

#### Reverse mapping dictionary

In [ ]:
idx2cate = dict(enumerate(cates))
idx2cate

In [ ]:
test_predict = np.vectorize(lambda x:idx2cate[x])(result_idx).tolist()

In [ ]:
test_predict

### Understanding Multi-Layer Percetron (simplist deep neural network)

I found hands on entertainment a lot better than I draw overwhelming lines and dots on the board. So we can try this [google neural network playground](https://playground.tensorflow.org/)

Also the [explain from 3blue 1brown](https://www.youtube.com/watch?v=aircAruvnKk) is very intuitive and brief. 

## Extras: The pytorch version of the code

In [16]:
import torch
from torch import nn
from torch.nn import functional as F

What extra libs you need for requirements

```
pytorch == 0.4.0
ray
```

"ray" is my [own ml/dl tool github repo](https://github.com/raynardj/ray)

You can gitclone it to your ```lib/python3.6/site-packages/``` folder

### Build up model

In [17]:
class mlp(nn.Module):
    def __init__(self):
        super(mlp,self).__init__()
        self.fc1 = nn.Linear(561,1000)
        self.fc2 = nn.Linear(1000,1000)
        self.fc3 = nn.Linear(1000,1000)
        self.fc4 = nn.Linear(1000,len(cates))
    
    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x,p=0.3)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x,p=0.3)
        x = self.fc3(x)
        x = F.relu(x)
        x = F.dropout(x,p=0.3)
        x = self.fc4(x)
        x = F.softmax(x)
        return x

In [18]:
model = mlp()
model

mlp(
  (fc1): Linear(in_features=561, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=1000, bias=True)
  (fc4): Linear(in_features=1000, out_features=6, bias=True)
)

### Prepare the data generator

In [19]:
from torch.utils.data.dataset import TensorDataset

In [20]:
trn_slice = np.random.rand(len(train_x_arr))>.7

In [21]:
ds_train = TensorDataset(torch.from_numpy(train_x_arr[trn_slice].astype(np.float)),
                        torch.from_numpy(train_y_arr[trn_slice].astype(np.float)),)

ds_valid = TensorDataset(torch.from_numpy(train_x_arr[~trn_slice].astype(np.float)),
                        torch.from_numpy(train_y_arr[~trn_slice].astype(np.float)),)

In [22]:
from torch.utils.data import DataLoader

In [23]:
train_ld = DataLoader(ds_train,batch_size = 256, shuffle = True)
valid_ld = DataLoader(ds_valid,batch_size = 256, shuffle = True)

#### Test generator

In [24]:
next(iter(train_ld))

[tensor([[ 0.2820, -0.0153, -0.1100,  ...,  0.3620, -0.3660, -0.6090],
         [ 0.2680, -0.0281, -0.1190,  ..., -0.6900,  0.3080,  0.0615],
         [ 0.3580,  0.0543, -0.0047,  ..., -0.7110,  0.2920,  0.0657],
         ...,
         [ 0.3460, -0.0371, -0.1010,  ..., -0.6970,  0.1240, -0.1990],
         [ 0.2790, -0.0180, -0.1090,  ..., -0.6080, -0.1610, -0.0717],
         [ 0.2760, -0.0170, -0.1090,  ..., -0.8120,  0.1650, -0.0987]], dtype=torch.float64),
 tensor([ 0.,  5.,  5.,  0.,  4.,  4.,  4.,  3.,  5.,  5.,  3.,  1.,
          5.,  3.,  3.,  3.,  2.,  2.,  3.,  2.,  2.,  2.,  5.,  3.,
          3.,  1.,  5.,  5.,  0.,  0.,  1.,  5.,  2.,  0.,  0.,  0.,
          0.,  2.,  1.,  0.,  2.,  4.,  1.,  2.,  5.,  5.,  0.,  4.,
          2.,  2.,  5.,  0.,  0.,  0.,  4.,  5.,  3.,  1.,  2.,  1.,
          2.,  5.,  5.,  2.,  1.,  1.,  4.,  0.,  1.,  0.,  0.,  1.,
          4.,  1.,  4.,  5.,  5.,  3.,  3.,  5.,  5.,  1.,  5.,  2.,
          5.,  4.,  1.,  5.,  4.,  0.,  5.,  0.,  4., 

### Wrap up trainer

In [25]:
from ray.matchbox import Trainer
from torch.optim import Adam

In [26]:
trainer = Trainer(ds_train,val_dataset=ds_valid,batch_size=256)

In [27]:
optimizer = Adam(model.parameters(),lr = 1e-3)
loss_func = nn.CrossEntropyLoss()

In [28]:
# define a training step
def action(*args,**kwargs):
    x,y = args[0]
    model.zero_grad()
    
    # forward pass calculation
    y_ = model(x.float())
    loss = loss_func(y_,y.long())
    
    # backward propagation (calculate the gradients)
    loss.backward()
    optimizer.step()
    return {"loss":loss.item()}

def val_action(*args,**kwargs):
    x,y = args[0]
    
    # forward pass calculation
    y_ = model(x.float())
    loss = loss_func(y_,y.long())
    return {"loss":loss.item()}
trainer.action = action
trainer.val_action = val_action

In [29]:
trainer.train(15)

😎[val_ep_14_i_9]	loss	1.173: 100%|██████████| 10/10 [00:00<00:00, 29.88it/s]
